# Using GPT-5 - OpenAI API Guide

## Introduction to OpenAI's Most Intelligent Model

GPT-5 is OpenAI's most advanced reasoning model, specifically trained for:
- **Code generation**, bug fixing, and refactoring
- **Instruction following** with high accuracy
- **Long context** handling and **tool calling** for agentic tasks

This notebook demonstrates GPT-5's key features with practical code examples.

## Setup

Install the OpenAI Python client if you haven't already:

In [ ]:
# Install OpenAI client
!pip install openai

In [ ]:
from openai import OpenAI
import os

# Initialize client
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")  # Set your API key as environment variable
)

## GPT-5 Model Variants

| Model | Best For | Trade-offs |
|-------|----------|------------|
| **`gpt-5`** | Complex reasoning, broad knowledge, code-heavy tasks | Highest capability, higher latency |
| **`gpt-5-mini`** | Cost-optimized reasoning and chat | Balanced speed, cost, and capability |
| **`gpt-5-nano`** | High-throughput, simple tasks | Fastest, most cost-effective |

## Quickstart: Fast Responses with Low Reasoning

For faster, lower-latency responses similar to GPT-4.1, use **low reasoning effort** and **low verbosity**:

In [ ]:
# Fast response with minimal reasoning
result = client.responses.create(
    model="gpt-5",
    input="Write a haiku about code.",
    reasoning={"effort": "low"},
    text={"verbosity": "low"},
)

print("Output:", result.output_text)
print("\nReasoning tokens used:", len(result.reasoning_text.split()) if hasattr(result, 'reasoning_text') else 'N/A')

## Reasoning Effort Control

GPT-5 supports four reasoning levels: `minimal`, `low`, `medium`, `high`

- **`minimal`**: Fastest time-to-first-token, best for coding & instruction following
- **`low`**: Quick responses with light reasoning
- **`medium`**: Default, balanced reasoning (similar to o3)
- **`high`**: Most thorough reasoning for complex problems

In [ ]:
# Minimal reasoning for fastest response
response_minimal = client.responses.create(
    model="gpt-5",
    input="Write a Python function to check if a number is prime.",
    reasoning={"effort": "minimal"}
)

print("Minimal Reasoning Output:")
print(response_minimal.output_text)

In [ ]:
# High reasoning for complex problems
response_high = client.responses.create(
    model="gpt-5",
    input="How much gold would it take to coat the Statue of Liberty in a 1mm layer? Show your calculations.",
    reasoning={"effort": "high"}
)

print("High Reasoning Output:")
print(response_high.output_text)

## Verbosity Control

Control output length with `verbosity` parameter:
- **`low`**: Concise answers, minimal code comments
- **`medium`**: Balanced explanations (default)
- **`high`**: Thorough explanations, detailed code documentation

In [ ]:
# Low verbosity for concise responses
response_concise = client.responses.create(
    model="gpt-5",
    input="Generate a SQL query to find the top 5 customers by total purchase amount.",
    text={"verbosity": "low"}
)

print("Concise Output:")
print(response_concise.output_text)

In [ ]:
# High verbosity for detailed explanations
response_detailed = client.responses.create(
    model="gpt-5",
    input="Explain how async/await works in JavaScript.",
    text={"verbosity": "high"}
)

print("Detailed Output:")
print(response_detailed.output_text[:500] + "...")  # Truncated for display

## Custom Tools: Freeform Text Inputs

GPT-5 introduces **custom tools** that accept raw text instead of structured JSON. Perfect for:
- Executing code snippets
- SQL queries
- Shell commands
- Configuration files

In [ ]:
# Define a custom tool for code execution
response_with_tool = client.responses.create(
    model="gpt-5",
    input="Use the code_exec tool to calculate the factorial of 10.",
    tools=[
        {
            "type": "custom",
            "name": "code_exec",
            "description": "Executes arbitrary Python code and returns the result"
        }
    ]
)

print("Tool Call Generated:")
if hasattr(response_with_tool, 'tool_calls'):
    for tool_call in response_with_tool.tool_calls:
        print(f"Tool: {tool_call.name}")
        print(f"Input: {tool_call.input}")

## Context-Free Grammar (CFG) Constraints

Constrain custom tool outputs to specific syntax using Lark grammars:

In [ ]:
# Example: SQL query tool with grammar constraints
sql_grammar = """
start: select_stmt
select_stmt: "SELECT" columns "FROM" table [where_clause] [order_clause] [limit_clause]
columns: "*" | column ("," column)*
column: WORD
table: WORD
where_clause: "WHERE" condition
condition: column operator value
operator: "=" | ">" | "<" | ">=" | "<=" | "LIKE"
value: STRING | NUMBER
order_clause: "ORDER BY" column ["ASC" | "DESC"]
limit_clause: "LIMIT" NUMBER

%import common.WORD
%import common.STRING
%import common.NUMBER
"""

response_sql = client.responses.create(
    model="gpt-5",
    input="Create a SQL query to find users named John ordered by registration date.",
    tools=[
        {
            "type": "custom",
            "name": "sql_query",
            "description": "Generates SQL queries",
            "grammar": sql_grammar
        }
    ]
)

print("Constrained SQL Output:")
if hasattr(response_sql, 'tool_calls'):
    print(response_sql.tool_calls[0].input)

## Allowed Tools: Selective Tool Access

Define a full toolkit but restrict which tools can be used in specific contexts:

In [ ]:
# Define multiple tools but restrict usage
all_tools = [
    {"type": "function", "name": "get_weather", "description": "Get current weather"},
    {"type": "function", "name": "search_docs", "description": "Search documentation"},
    {"type": "function", "name": "run_tests", "description": "Execute test suite"},
    {"type": "function", "name": "deploy_code", "description": "Deploy to production"}
]

# Only allow safe operations
response_restricted = client.responses.create(
    model="gpt-5",
    input="What's the weather like and can you search for React hooks documentation?",
    tools=all_tools,
    tool_choice={
        "type": "allowed_tools",
        "mode": "auto",  # Model decides which to use
        "tools": [
            {"type": "function", "name": "get_weather"},
            {"type": "function", "name": "search_docs"}
        ]
    }
)

print("Model can only access: get_weather, search_docs")
print("Model cannot access: run_tests, deploy_code")

## Tool Preambles for Transparency

Enable preambles to see GPT-5's reasoning before tool calls:

In [ ]:
# Enable preambles for tool transparency
response_preamble = client.responses.create(
    model="gpt-5",
    input="Before you call a tool, explain why you are calling it. Now search for information about Python decorators.",
    tools=[
        {"type": "function", "name": "search_docs", "description": "Search documentation"}
    ]
)

print("Response with preamble:")
print(response_preamble.output_text)

## Migration Guide

### From Other Models to GPT-5

| From Model | Migrate To | Recommended Settings |
|------------|------------|---------------------|
| **o3** | `gpt-5` | `reasoning.effort: "medium"` or `"high"` |
| **gpt-4.1** | `gpt-5` | `reasoning.effort: "minimal"` or `"low"` |
| **o4-mini** | `gpt-5-mini` | Default settings with prompt tuning |
| **gpt-4.1-nano** | `gpt-5-nano` | Default settings with prompt tuning |

### ⚠️ Important: Unsupported Parameters

GPT-5 does **NOT** support:
- `temperature`
- `top_p` 
- `logprobs`

Use GPT-5-specific controls instead:
- `reasoning: {effort: ...}`
- `text: {verbosity: ...}`
- `max_output_tokens`

## Responses API vs Chat Completions

### Key Advantage: Chain of Thought (CoT) Persistence

The Responses API passes reasoning between turns, resulting in:
- Improved intelligence
- Fewer reasoning tokens generated
- Higher cache hit rates
- Lower latency

In [ ]:
# Multi-turn conversation with CoT persistence
first_response = client.responses.create(
    model="gpt-5",
    input="Let's solve a complex problem. What's the optimal way to implement a LRU cache in Python?",
    reasoning={"effort": "medium"}
)

print("First response:", first_response.output_text[:200] + "...")

# Continue conversation, passing previous response ID
follow_up = client.responses.create(
    model="gpt-5",
    input="Now add thread-safety to that implementation.",
    previous_response_id=first_response.id  # Passes CoT from previous turn
)

print("\nFollow-up (with CoT context):", follow_up.output_text[:200] + "...")

## Best Practices

### 1. Choose the Right Model
- **`gpt-5`**: Complex reasoning, coding, multi-step tasks
- **`gpt-5-mini`**: General chat, moderate complexity
- **`gpt-5-nano`**: Simple tasks, high throughput

### 2. Optimize for Your Use Case
- **Speed Priority**: Use `minimal` reasoning + `low` verbosity
- **Quality Priority**: Use `high` reasoning + `high` verbosity
- **Balanced**: Use defaults (`medium` for both)

### 3. Leverage New Features
- **Custom Tools**: For code execution, SQL, configs
- **Allowed Tools**: For safety and predictability
- **Preambles**: For debugging and transparency

### 4. Use Responses API for Multi-turn
- Always pass `previous_response_id` for context
- Reduces re-reasoning and improves coherence

## Practical Example: Building a Code Assistant

In [ ]:
def code_assistant(task, code_context=None, optimize_for="balanced"):
    """
    GPT-5 powered code assistant with configurable optimization.
    
    Args:
        task: What you want the assistant to do
        code_context: Existing code to work with
        optimize_for: "speed", "quality", or "balanced"
    """
    
    # Configure based on optimization preference
    settings = {
        "speed": {"reasoning": {"effort": "minimal"}, "text": {"verbosity": "low"}},
        "quality": {"reasoning": {"effort": "high"}, "text": {"verbosity": "high"}},
        "balanced": {"reasoning": {"effort": "medium"}, "text": {"verbosity": "medium"}}
    }
    
    config = settings.get(optimize_for, settings["balanced"])
    
    # Build the prompt
    prompt = task
    if code_context:
        prompt = f"Task: {task}\n\nExisting code:\n```python\n{code_context}\n```"
    
    # Call GPT-5
    response = client.responses.create(
        model="gpt-5",
        input=prompt,
        **config
    )
    
    return response.output_text

# Example usage
result = code_assistant(
    task="Add error handling and logging to this function",
    code_context="""def divide(a, b):
    return a / b""",
    optimize_for="quality"
)

print(result)

## Conclusion

GPT-5 represents a significant leap in AI reasoning capabilities. Key takeaways:

1. **Use the Responses API** for multi-turn conversations to leverage CoT persistence
2. **Configure reasoning and verbosity** based on your latency/quality requirements  
3. **Leverage new features** like custom tools and allowed tools for better control
4. **Choose the right model variant** (gpt-5, gpt-5-mini, gpt-5-nano) for your use case
5. **Migrate gradually** using the recommended settings for your current model

For more information:
- [GPT-5 System Card](https://openai.com/index/gpt-5-system-card/)
- [GPT-5 Prompting Guide](https://cookbook.openai.com/examples/gpt-5/gpt-5_prompting_guide)
- [GPT-5 Frontend Development](https://cookbook.openai.com/examples/gpt-5/gpt-5_frontend)
- [API Documentation](https://platform.openai.com/docs/guides/latest-model)